## Lasso Regression

We will try to fit the dataset with a Lasso Regression model. The steps are 

- Implement the Shooting Algorithm
 * allow for random or non-random order for the coordinates
 * allow for initial weights all zero or the corresponding solution to Ridge Regression
- Determine a class for the model supporting methods
 * fit 
 * predict 
 * score
- Tune hyperparameters
 * Search for hyperparameters through trial and error
 * Use upper bound on hyperparameter with warm start
- Plot the distributions of weight on the features 
 * Does Lasso Regression give us sparsity
- Threshold the values to compare zero/non-zero against the weights of the target function
- Implement Projected Gradient Descent 
 * Compare to Shooting Algorithm

In [9]:
import numpy as np
np.random.seed(42)
import pandas as pd
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.optimize import minimize

from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.metrics import confusion_matrix

from load_data import load_problem

PICKLE_PATH = 'lasso_data.pickle'

#### Dataset

In [10]:
#load data 

x_train, y_train, x_val, y_val, target_fn, coefs_true, featurize = load_problem(PICKLE_PATH)
X_train = featurize(x_train)
X_val = featurize(x_val)

In [ ]:
#Visualize training data

fig, ax = plt.subplots(figsize = (7,7))
ax.set_title("({0}, {1}) Design Matrix".format(X_train.shape[0], X_train.shape[1]))
ax.set_xlabel("Feature Index")
ax.set_ylabel("Example Number")
temp = ax.imshow(X_train, cmap=plt.cm.Purples, aspect="auto")
plt.colorbar(temp, shrink=0.5);

#### Coordinate Descent for Lasso Regression (Shooting Algorithm)

For the shooting algorithm, we need to compute the Lasso Regression objective for the stopping condition. Moreover we need a threshold function at each iteration along with the solution to Ridge Regression for initial weights.  

In [6]:
def soft_threshold(a, delta):
    ####
    # your code goes here
    ####
    
def compute_sum_sqr_loss(X, y, w):
    ####
    # your code goes here
    ####
    
def compute_lasso_objective(X, y, w, l1_reg=0):
    ####
    # your code goes here
    ####
    
def get_ridge_solution(X, y, l2_reg):
    ####
    # your code goes here
    ####

Remember that we should avoid loops in the implementation because we need to run the algorithm repeatedly for hyperparameter tuning.

Please see Lecture 4 notes for derivation of shooting algorithm.

In [7]:
def shooting_algorithm(X, y, w0=None, l1_reg = 1., max_num_epochs = 1000, min_obj_decrease=1e-8, random=False):
    if w0 is None:
        w = np.zeros(X.shape[1])
    else:
        w = np.copy(w0)
    d = X.shape[1]
    epoch = 0
    obj_val = compute_lasso_objective(X, y, w, l1_reg)
    obj_decrease = min_obj_decrease + 1.
    while (obj_decrease>min_obj_decrease) and (epoch<max_num_epochs):
        obj_old = obj_val
        # Cyclic coordinates descent
        coordinates = range(d)
        # Randomized coordinates descent
        if random:
            coordinates = np.random.permutation(d)
        for j in coordinates:
            ####
            # your code goes here
            ...
            ...
            ####
            
        obj_val = compute_lasso_objective(X, y, w, l1_reg)
        obj_decrease = obj_old - obj_val
        epoch += 1
    print("Ran for "+str(epoch)+" epochs. " + 'Lowest loss: ' + str(obj_val))
    return w

#### Class for Lasso Regression 

In [8]:
class LassoRegression(BaseEstimator, RegressorMixin):
    """ Lasso regression"""
    def __init__(self, l1_reg=1.0, randomized=False):
        if l1_reg < 0:
            raise ValueError('Regularization penalty should be at least 0.')
        self.l1_reg = l1_reg
        self.randomized = randomized


    def fit(self, X, y, max_epochs = 1000, coef_init=None):
        # convert y to 1-dim array, in case we're given a column vector
        y = y.reshape(-1)
        if coef_init is None:
            coef_init = get_ridge_solution(X,y, self.l1_reg)
        
        ####
        # your code goes here
        ...
        ...
        ####
        return self

    def predict(self, X, y=None):
        try:
            getattr(self, "w_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")

        return np.dot(X, self.w_)

    def score(self, X, y):
        try:
            getattr(self, "w_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")

        return compute_sum_sqr_loss(X, y, self.w_)/len(y)

We can compare to the `sklearn` implementation.

In [9]:
def compare_our_lasso_with_sklearn(X_train, y_train, l1_reg=1):
    
    # Fit with sklearn -- need to divide l1_reg by 2*sample size, since they
    # use a slightly different objective function.
    n = X_train.shape[0]
    sklearn_lasso = Lasso(alpha=l1_reg/(2*n), fit_intercept=False, normalize=False)
    sklearn_lasso.fit(X_train, y_train)
    sklearn_lasso_coefs = sklearn_lasso.coef_
    sklearn_lasso_preds = sklearn_lasso.predict(X_train)

    # Now run our lasso regression and compare the coefficients to sklearn's
    
    ####
    # your code goes here
    ...
    ...
    ####

    # Let's compare differences in predictions
    print("Hoping this is very close to 0 (predictions): {}".format( np.mean((sklearn_lasso_preds - lasso_regression_preds)**2)))
    # Let's compare differences parameter values
    print("Hoping this is very close to 0: {}".format(np.sum((our_coefs - sklearn_lasso_coefs)**2)))

In [36]:
compare_our_ridge_with_sklearn(X_train, y_train, l2_reg=1.5)

Hoping this is very close to 0:4.6903718660670966e-11


#### Grid Search to Tune Hyperparameter

Now let's use sklearn to help us do hyperparameter tuning
GridSearchCv.fit by default splits the data into training and
validation itself; we want to use our own splits, so we need to stack our
training and validation sets together, and supply an index
(validation_fold) to specify which entries are train and which are
validation.

In [ ]:
def do_grid_search_lasso(X_train, y_train, X_val, y_val):
    ####
    ## your code goes here
    ...
    ...
    ####

In [43]:
grid, results = do_grid_search_ridge(X_train, y_train, X_val, y_val)

In [44]:
results

,param_l2reg,mean_test_score,mean_train_score
0,0.000001,0.172579,0.006752
1,0.000010,0.172464,0.006752
2,0.000100,0.171345,0.006774
3,0.001000,0.162705,0.008285
4,0.010000,0.141887,0.032767
5,0.100000,0.144566,0.094953
6,1.000000,0.171068,0.197694
7,1.300000,0.179521,0.216591
8,1.600000,0.187993,0.233450
9,1.900000,0.196361,0.248803


In [ ]:
# Plot validation performance vs regularization parameter

fig, ax = plt.subplots(figsize = (8,6))
ax.grid()
ax.set_title("Validation Performance vs L1 Regularization")
ax.set_xlabel("L1-Penalty Regularization Parameter")
ax.set_ylabel("Mean Squared Error")

####
## your code goes here
...
...
####

ax.text(0.005,0.17,"Best parameter {0}".format(grid.best_params_['l1reg']), fontsize = 12);

#### Comparing to the Target Function

Let's plot prediction functions and compare coefficients for several fits and the target function.


Let's create a list of dicts called `pred_fns`. Each dict has a "name" key and a
"preds" key. The value corresponding to the "preds" key is an array of
predictions corresponding to the input vector x. x_train and y_train are
the input and output values for the training data

In [57]:
pred_fns = []
x = np.sort(np.concatenate([np.arange(0,1,.001), x_train]))

pred_fns.append({"name": "Target Function", "coefs": coefs_true, "preds": target_fn(x)})

l1regs = [0.1, grid.best_params_['l1reg'], 1]
X = featurize(x)
for l1reg in l1regs:
    lasso_regression_estimator = LassoRegression(l1reg=l1reg)
    lasso_regression_estimator.fit(X_train, y_train)
    name = "Lasso with L1Reg="+str(l1reg)

    ####
    ## your code goes here
    ...
    ...
    ####    

In [62]:
def plot_prediction_functions(x, pred_fns, x_train, y_train, legend_loc="best"):

	fig, ax = plt.subplots(figsize = (12,8))
	ax.set_xlabel('Input Space: [0,1)')
	ax.set_ylabel('Action/Outcome Space')
	ax.set_title("Prediction Functions")
	plt.scatter(x_train, y_train, color="k", label='Training data')
	for i in range(len(pred_fns)):
		ax.plot(x, pred_fns[i]["preds"], label=pred_fns[i]["name"])
	legend = ax.legend(loc=legend_loc, shadow=True)
	return fig

In [ ]:
plot_prediction_functions(x, pred_fns, x_train, y_train, legend_loc="best");

#### Visualizing the Weights

Using `pred_fns` let's try to see how sparse the weights are... 

In [107]:
def compare_parameter_vectors(pred_fns):

	fig, axs = plt.subplots(len(pred_fns),1, sharex=True, figsize = (20,20))
	num_ftrs = len(pred_fns[0]["coefs"])
	for i in range(len(pred_fns)):
		title = pred_fns[i]["name"]
		coef_vals = pred_fns[i]["coefs"]
		axs[i].bar(range(num_ftrs), coef_vals, color = "tab:purple")
		axs[i].set_xlabel('Feature Index')
		axs[i].set_ylabel('Parameter Value')
		axs[i].set_title(title)

	fig.subplots_adjust(hspace=0.4)
	return fig

In [ ]:
compare_parameter_vectors(pred_fns);

#### Continuation Method

We compute the largest value of $\lambda$ for which the weights can be nonzero.

In [12]:
def get_lambda_max_no_bias(X, y):
    return 2 * np.max(np.abs(np.dot(y, X)))

Use homotopy method to compute regularization path for LassoRegression.

In [12]:
class LassoRegularizationPath:
    def __init__(self, estimator, tune_param_name):
        self.estimator = estimator
        self.tune_param_name = tune_param_name

    def fit(self, X, y, reg_vals, coef_init=None, warm_start=True):
        # reg_vals is a list of regularization parameter values to solve for.
        # Solutions will be found in the order given by reg_vals.

        #convert y to 1-dim array, in case we're given a column vector
        y = y.reshape(-1)

        if coef_init is not None:
            coef_init = np.copy(coef_init)

        self.results = []
        for reg_val in reg_vals:
            estimator = clone(self.estimator)

            ####
            ## your code goes here
            ...
            ...
            ####
            
            self.results.append({"reg_val":reg_val, "estimator":estimator})

        return self

    def predict(self, X, y=None):
        predictions = []
        for i in range(len(self.results)):
            preds = self.results[i]["estimator"].predict(X)
            reg_val = self.results[i]["reg_val"]
            predictions.append({"reg_val":reg_val, "preds":preds})
        return predictions

    def score(self, X, y=None):
        scores = []
        for i in range(len(self.results)):
            score = self.results[i]["estimator"].score(X, y)
            reg_val = self.results[i]["reg_val"]
            scores.append({"reg_val":reg_val, "score":score})
        return scores

In [ ]:
def do_grid_search_homotopy(X_train, y_train, X_val, y_val,
                            reg_vals=None, w0=None):
    if reg_vals is None:
        lambda_max = get_lambda_max_no_bias(X_train, y_train)
        reg_vals = [lambda_max * (.8**n) for n in range(0, 30)]
    
    ####
    ## your code goes here
    ...
    ...
    ####
    
    estimator = LassoRegression()
    lasso_reg_path_estimator = LassoRegularizationPath(estimator, tune_param_name="l1_reg")
    lasso_reg_path_estimator.fit(X_train, y_train,
                                 reg_vals=reg_vals[:], coef_init=w0,
                                 warm_start=True)
    
    return lasso_reg_path_estimator, reg_vals

In [ ]:
lasso_reg_path_estimator, reg_vals = do_grid_search_homotopy(X_train, 
                                                             y_train, 
                                                             X_val, 
                                                             y_val, 
                                                             None)

#### Projected SGD

In [ ]:
def projection_SGD_split(X, y, theta_positive_0, theta_negative_0, lambda_reg = 1.0, alpha = 0.1, num_iter = 1000):
    m, n = X.shape
    theta_positive = np.zeros(n)
    theta_negative = np.zeros(n)
    theta_positive[0:n] = theta_positive_0
    theta_negative[0:n] = theta_negative_0
    times = 0
    theta = theta_positive - theta_negative
    loss = compute_sum_sqr_loss(X, y, theta)
    loss_change = 1.
    while (loss_change>1e-6) and (times<num_iter):
        loss_old = loss
        for i in range(m):
            ####
            ## your code goes here
            ...
            ...
            ####
        loss = compute_sum_sqr_loss(X, y, theta)
        loss_change = np.abs(loss - loss_old)
        times +=1

    print('(SGD) Ran for {} epochs. Loss:{} Lambda: {}'.format(times,loss,lambda_reg))
    return theta

In [ ]:
x_training, y_training, x_validation, y_validation, target_fn, coefs_true, featurize = load_problem(PICKLE_PATH)
X_training = featurize(x_training)
X_validation = featurize(x_validation)
D = X_training.shape[1]

lambda_max = get_lambda_max_no_bias(x_training, y_training)
reg_vals = [lambda_max * (.6**n) for n in range(15, 25)]

loss_SGD_list = []
loss_shooting = []
loss_GD_list = []
    
for lambda_value in reg_vals:
    ####
    ## your code goes here
    ...
    ...
    ####   

In [ ]:
# Plot validation performance vs regularization parameter

fig, ax = plt.subplots(figsize = (8,6))
ax.grid()
ax.set_title("Validation Performance vs L1 Regularization")
ax.set_xlabel("L1-Penalty Regularization Parameter")
ax.set_ylabel("Mean Squared Error")

plt.semilogx(reg_vals, loss_shooting, label = 'Shooting Method')
plt.semilogx(reg_vals, loss_SGD_list, label = 'Projection SGD')
plt.legend(loc='upper left')
plt.show();

In [ ]:
# Report the best 

lambda_best_SGD = reg_vals[np.argmin(loss_SGD_list)]
theta_lasso_SGD_best = projection_SGD_split(X_training, y_training, theta_positive_ini, theta_negative_ini, lambda_reg=lambda_best_SGD, alpha = 0.01)
print('Best lambda for SGD is {0} with loss {1}'.format(lambda_best_SGD, np.min(loss_SGD_list)))